In [44]:
from google.colab import drive
drive.mount('/content/gdrive') 
%cd "/content/gdrive/My Drive/Econ488Paper" 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Econ488Paper


In [45]:
import pandas as pd
import numpy as np 
from datetime import datetime
from numpy.linalg import norm 
import matplotlib.pyplot as plt 
               
import warnings                  
warnings.filterwarnings("ignore") 

In [46]:
data = pd.read_csv("./final_data/final_cases.csv")  
mask_data = pd.read_csv("./final_data/cleaned_mask_data.csv") 

In [63]:
new = pd.read_csv("./demographics/cc-est2019-agesex-01.csv") 

In [64]:
new.head(2)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,...,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,50,1,1,Alabama,Autauga County,1,54571,26569,28002,3579,1866,1713,7418,3747,3671,3616,1842,1774,4617,2335,2282,41804,20046,21758,39958,19114,20844,22100,10867,11233,14730,7115,7615,14065,6843,7222,6546,2821,3725,3579,...,1614,3330,1594,1736,4157,2004,2153,4086,1974,2112,4332,2174,2158,3873,1866,2007,3083,1524,1559,2777,1279,1498,2277,1014,1263,1736,807,929,1251,546,705,731,295,436,551,159,392,37.0,35.9,37.9
1,50,1,1,Alabama,Autauga County,2,54597,26584,28013,3584,1869,1715,7425,3752,3673,3618,1843,1775,4617,2335,2282,41817,20052,21765,39970,19120,20850,22116,10874,11242,14744,7121,7623,14067,6845,7222,6542,2819,3723,3584,...,1616,3333,1595,1738,4160,2006,2154,4089,1974,2115,4334,2176,2158,3873,1866,2007,3083,1524,1559,2777,1279,1498,2277,1014,1263,1734,806,928,1250,546,704,730,294,436,551,159,392,37.0,35.9,37.8


In [65]:
cols = "STATE COUNTY STNAME	CTYNAME	YEAR POPESTIMATE AGE1824_TOT AGE2529_TOT MEDIAN_AGE_TOT".strip().split()  

In [66]:
new = new[cols]
new.head(2) 

,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,AGE1824_TOT,AGE2529_TOT,MEDIAN_AGE_TOT
0,1,1,Alabama,Autauga County,1,54571,4617,3157,37.0
1,1,1,Alabama,Autauga County,2,54597,4617,3162,37.0


In [69]:
new.iloc[:, 6: 8] 

,AGE1824_TOT,AGE2529_TOT
0,4617,3157
1,4617,3162
2,4671,3191
3,4816,3216
4,4802,3203
...,...,...
799,1710,1301
800,1710,1253
801,1731,1274
802,1764,1276


In [70]:
new["pct_18_29"] = new.iloc[:, 6: 8].sum(axis = 1) / new.POPESTIMATE 
new.COUNTY = new.CTYNAME.map(lambda x: x[: -7])  
new.COUNTY += " " + new.STNAME 

new = new[["YEAR", 'COUNTY', "pct_18_29"]]    
new = new[new.YEAR == new.YEAR.unique().max()].drop_duplicates() 
new.head() 

,YEAR,COUNTY,pct_18_29
11,12,Autauga Alabama,0.147917
23,12,Baldwin Alabama,0.127324
35,12,Barbour Alabama,0.153326
47,12,Bibb Alabama,0.156024
59,12,Blount Alabama,0.139211


In [71]:
final = new.copy() 

In [72]:
for i in range(2, 57): 
    i_str = str(i) 

    if len(i_str) == 1: 
        filepath = "./demographics/cc-est2019-agesex-0{}.csv".format(i_str) 
    else: 
        filepath = "./demographics/cc-est2019-agesex-{}.csv".format(i_str) 
    try: 
        new = pd.read_csv(filepath) 

        cols = "STATE COUNTY STNAME	CTYNAME	YEAR POPESTIMATE AGE1824_TOT AGE2529_TOT".strip().split()   

        new["pct_18_29"] = new.iloc[:, 6: 8].sum(axis = 1) / new.POPESTIMATE 
        new.COUNTY = new.CTYNAME.map(lambda x: x[: -7])   
        new.COUNTY += " " + new.STNAME 

        new = new[["YEAR", 'COUNTY', "pct_18_29"]]    
        new = new[new.YEAR == new.YEAR.unique().max()].drop_duplicates()

        final = final.append(new)  
    except: 
        continue   


In [73]:
final.shape

(3109, 3)

In [83]:
county_set = set(data.county.unique()).intersection(set(final.COUNTY.unique())) 

In [75]:
final.head()

,YEAR,COUNTY,pct_18_29
11,12,Autauga Alabama,0.147917
23,12,Baldwin Alabama,0.127324
35,12,Barbour Alabama,0.153326
47,12,Bibb Alabama,0.156024
59,12,Blount Alabama,0.139211


In [84]:
temp_data = data[data.county.isin(county_set)] 

temp_mask_data = mask_data[mask_data.COUNTYFP.isin(set(temp_data.fips.unique()))]  

temp_final = final[final.COUNTY.isin(set(temp_data.county.unique()))] 

In [85]:
len(temp_data.county.unique())

2971

In [86]:
len(temp_mask_data.COUNTYFP.unique()) 

2971

In [87]:
len(temp_final.COUNTY.unique())

2971

In [88]:
temp_final.head()

,YEAR,COUNTY,pct_18_29
11,12,Autauga Alabama,0.147917
23,12,Baldwin Alabama,0.127324
35,12,Barbour Alabama,0.153326
47,12,Bibb Alabama,0.156024
59,12,Blount Alabama,0.139211


In [89]:
temp_final.to_csv("age_data.csv") 

In [82]:
# temp_data.to_csv("final_cases.csv")        
# temp_mask_data.to_csv("cleaned_mask_data.csv") 
# temp_final.to_csv("demographic_data.csv")  

In [ ]:
new = pd.read_csv("./age_race_data/cc-est2019-alldata.csv", encoding='latin-1') 
new.shape

(716376, 80)

In [ ]:
new.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,21295,22002,4559,5130,119,139,200,284,29,18,367,429,21633,22391,4704,5306,277,314,300,409,42,37,25875,27386,20709,21485,4512,5091,103,115,194,280,13,9,344,406,21026,21853,4647,5258,251,282,291,398,23,27,694,616,586,517,47,39,16,24,6,4,16,9,23,23,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,1411,1316,362,317,5,3,13,15,1,0,74,62,1479,1368,405,362,23,18,34,28,3,1,1778,1651,1337,1260,356,313,2,2,13,15,0,0,70,61,1402,1312,396,357,19,17,34,28,1,0,88,62,74,56,6,4,3,1,0,0,1,0,4,1,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,1521,1526,399,374,14,8,17,21,1,3,49,58,1570,1583,425,403,27,19,32,42,3,4,1933,1916,1460,1465,398,372,12,2,17,21,0,3,46,53,1506,1517,423,400,25,12,30,39,1,4,68,74,61,61,1,2,2,6,0,0,1,0,3,5,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,1658,1620,431,406,15,12,23,18,4,1,40,62,1694,1681,453,436,29,27,32,37,4,5,2105,2055,1613,1570,421,403,12,9,22,18,3,0,34,55,1643,1624,440,429,24,22,30,36,3,4,66,64,45,50,10,3,3,3,1,0,1,1,6,7,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,1628,1585,502,424,12,7,25,14,4,2,42,45,1664,1624,525,444,23,20,39,31,6,5,2153,2026,1580,1543,495,420,12,5,23,14,1,1,42,43,1616,1580,518,439,23,18,37,30,3,4,60,51,48,42,7,4,0,2,2,0,3,1,0,2,48,44,7,5,0,2,2,1,3,1


In [ ]:
new = new[new.YEAR == new.YEAR.unique().max()]  
new.shape

(59698, 80)

In [ ]:
new.CTYNAME = new.CTYNAME.map(lambda x: x[: -7]) 
new.head() 

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
209,50,1,1,Alabama,Autauga,12,0,55869,27092,28777,20878,21729,5237,6000,121,145,286,370,32,26,538,507,21372,22191,5514,6270,274,301,441,509,55,49,26208,27990,20138,21077,5171,5927,105,138,282,364,20,20,492,464,20594,21504,5425,6177,234,274,426,490,39,38,884,787,740,652,66,73,16,7,4,6,12,6,46,43,778,687,89,93,40,27,15,19,16,11
210,50,1,1,Alabama,Autauga,12,1,3277,1713,1564,1249,1117,339,344,4,6,24,20,3,3,94,74,1337,1189,409,396,13,15,46,33,7,6,1627,1504,1180,1072,334,340,3,6,23,19,2,3,85,64,1261,1136,399,390,7,10,43,29,4,3,86,60,69,45,5,4,1,0,1,1,1,0,9,10,76,53,10,6,6,5,3,4,3,3
211,50,1,1,Alabama,Autauga,12,2,3465,1787,1678,1287,1191,388,368,10,9,17,25,0,2,85,83,1369,1270,450,430,21,20,35,38,2,5,1699,1608,1210,1134,388,359,7,8,16,25,0,1,78,81,1286,1211,448,420,13,18,33,38,1,4,88,70,77,57,0,9,3,1,1,0,0,1,7,2,83,59,2,10,8,2,2,0,1,1
212,50,1,1,Alabama,Autauga,12,3,3851,1977,1874,1441,1350,442,418,3,9,18,25,2,4,71,68,1510,1415,490,465,10,14,35,49,6,6,1883,1795,1362,1285,435,409,3,9,17,24,0,3,66,65,1426,1348,479,453,8,12,34,47,4,5,94,79,79,65,7,9,0,0,1,1,2,1,5,3,84,67,11,12,2,2,1,2,2,1
213,50,1,1,Alabama,Autauga,12,4,3659,1854,1805,1341,1334,434,400,5,0,21,13,5,3,48,55,1386,1383,463,432,12,12,33,31,9,4,1791,1731,1291,1272,429,397,4,0,21,13,3,3,43,46,1331,1315,456,426,8,7,33,27,6,4,63,74,50,62,5,3,1,0,0,0,2,0,5,9,55,68,7,6,4,5,0,4,3,0


In [ ]:
new.CTYNAME += " " + new.STNAME 
new.head(2)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
209,50,1,1,Alabama,Autauga Alabama,12,0,55869,27092,28777,20878,21729,5237,6000,121,145,286,370,32,26,538,507,21372,22191,5514,6270,274,301,441,509,55,49,26208,27990,20138,21077,5171,5927,105,138,282,364,20,20,492,464,20594,21504,5425,6177,234,274,426,490,39,38,884,787,740,652,66,73,16,7,4,6,12,6,46,43,778,687,89,93,40,27,15,19,16,11
210,50,1,1,Alabama,Autauga Alabama,12,1,3277,1713,1564,1249,1117,339,344,4,6,24,20,3,3,94,74,1337,1189,409,396,13,15,46,33,7,6,1627,1504,1180,1072,334,340,3,6,23,19,2,3,85,64,1261,1136,399,390,7,10,43,29,4,3,86,60,69,45,5,4,1,0,1,1,1,0,9,10,76,53,10,6,6,5,3,4,3,3


In [ ]:
cols = "CTYNAME TOT_POP WA_MALE WA_FEMALE BA_MALE BA_FEMALE IA_MALE IA_FEMALE AA_MALE AA_FEMALE NA_MALE NA_FEMALE".strip().split()   

In [ ]:
temp = new[cols] 

temp.head(3)

,CTYNAME,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE
209,Autauga Alabama,55869,20878,21729,5237,6000,121,145,286,370,32,26
210,Autauga Alabama,3277,1249,1117,339,344,4,6,24,20,3,3
211,Autauga Alabama,3465,1287,1191,388,368,10,9,17,25,0,2


In [ ]:
temp = temp.drop_duplicates(subset = ["CTYNAME"])
temp.head()

,CTYNAME,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE
209,Autauga Alabama,55869,20878,21729,5237,6000,121,145,286,370,32,26
437,Baldwin Alabama,223234,94810,100388,9486,10107,903,839,932,1448,74,80
665,Barbour Alabama,24686,6389,5745,6311,5595,103,67,55,61,34,18
893,Bibb Alabama,22394,8766,8425,2941,1822,53,50,23,25,22,4
1121,Blount Alabama,57826,27258,28154,516,462,192,178,85,100,42,25


In [ ]:
temp["pct_white"] = (temp.WA_FEMALE + temp.WA_MALE) / temp.TOT_POP 
temp["pct_black"] = (temp.BA_FEMALE + temp.BA_MALE) / temp.TOT_POP 
temp["pct_american_indian"] = (temp.IA_FEMALE + temp.IA_MALE) / temp.TOT_POP 
temp["pct_asian"] = (temp.AA_FEMALE + temp.AA_MALE) / temp.TOT_POP 
temp["pct_pacific_island"] = (temp.NA_FEMALE + temp.NA_MALE) / temp.TOT_POP    

In [ ]:
temp.head()

,CTYNAME,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,pct_white,pct_black,pct_american_indian,pct_asian,pct_pacific_island
209,Autauga Alabama,55869,20878,21729,5237,6000,121,145,286,370,32,26,0.762623,0.201131,0.004761,0.011742,0.001038
437,Baldwin Alabama,223234,94810,100388,9486,10107,903,839,932,1448,74,80,0.874410,0.087769,0.007803,0.010661,0.000690
665,Barbour Alabama,24686,6389,5745,6311,5595,103,67,55,61,34,18,0.491534,0.482298,0.006886,0.004699,0.002106
893,Bibb Alabama,22394,8766,8425,2941,1822,53,50,23,25,22,4,0.767661,0.212691,0.004599,0.002143,0.001161
1121,Blount Alabama,57826,27258,28154,516,462,192,178,85,100,42,25,0.958254,0.016913,0.006399,0.003199,0.001159


In [ ]:
temp.columns

Index(['CTYNAME', 'TOT_POP', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE',
       'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE',
       'pct_white', 'pct_black', 'pct_american_indian', 'pct_asian',
       'pct_pacific_island'],
      dtype='object')

In [ ]:
temp = temp[['CTYNAME', 'pct_white', 'pct_black', 'pct_american_indian', 'pct_asian', 'pct_pacific_island']] 

In [ ]:
temp.head()

,CTYNAME,pct_white,pct_black,pct_american_indian,pct_asian,pct_pacific_island
209,Autauga Alabama,0.762623,0.201131,0.004761,0.011742,0.001038
437,Baldwin Alabama,0.874410,0.087769,0.007803,0.010661,0.000690
665,Barbour Alabama,0.491534,0.482298,0.006886,0.004699,0.002106
893,Bibb Alabama,0.767661,0.212691,0.004599,0.002143,0.001161
1121,Blount Alabama,0.958254,0.016913,0.006399,0.003199,0.001159


In [ ]:
temp.to_csv("demographics_race.csv") 